# Geolocalización con Information Value Personas

En esta notebook haremos un intento de geolocalización con los textos de los usuarios...

In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pandas as pd

df_train = pd.read_json("../data/geoloc/users_train.json")
df_test = pd.read_json("../data/geoloc/users_test.json")

Hagamos lo siguiente:

- Entrenemos con unigramas una regresión logística para 
- Luego probemos con los regionalismos

Primero, partamos en train, test

In [5]:
df_train.groupby("provincia").count()

,text
provincia,
buenosaires,337
catamarca,341
chaco,331
chubut,328
cordoba,317
corrientes,345
entrerios,338
formosa,286
jujuy,339


## Palabras precalculadas

Carguemos antes las palabras que sabemos que ocurren una cantidad razonable de veces

In [6]:
%%time
from contrastes.processing import build_dataframe_from_users
from contrastes.processing import preprocess_raw_df


#word_df = build_dataframe_from_users(row for index, row in df_train.iterrows())

word_df = pd.read_csv("train_word_df_filtered.csv", index_col=0)
word_df = preprocess_raw_df(word_df, filter_words=(10, 2))

CPU times: user 704 ms, sys: 68 ms, total: 772 ms
Wall time: 770 ms


/home/jmperez/projects/contrastes/contrastes/processing.py:185: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.columnas_palabras = cant_palabras
/home/jmperez/projects/contrastes/contrastes/processing.py:186: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.columnas_personas = cant_personas


La reg. logística será un softmax, así que elijo `multi_class='multinomial'`

In [7]:
from contrastes.lists import add_ival

add_ival(word_df, normalize=True)

Calculating information values...
Calculating ranks...


In [8]:
word_df.sort_values("rank_personas", ascending=True, inplace=True)

word_df.iloc[:10]

,buenosaires_ocurrencias,buenosaires_usuarios,catamarca_ocurrencias,catamarca_usuarios,chaco_ocurrencias,chaco_usuarios,chubut_ocurrencias,chubut_usuarios,cordoba_ocurrencias,cordoba_usuarios,...,tucuman_usuarios,cant_provincias,cant_palabra,cant_usuarios,ival_palabras,ival_personas,ival_palper,rank_palabras,rank_personas,rank_palper
ush,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,6,638.0,346.0,1.241734,1.612927,2.002826,37.0,1.0,4.0
poec,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,163.0,164.0,1.059592,1.584212,1.678618,117.5,2.0,18.0
chivil,332.0,79.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,332.0,158.0,1.207573,1.574520,1.901348,46.0,3.0,6.0
plottier,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,3,848.0,206.0,1.387099,1.570455,2.178376,14.0,4.0,3.0
chivilcoy,2331.0,125.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,6,2337.0,262.0,1.599973,1.569203,2.510682,4.0,5.0,1.0
vallerga,291.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,291.0,144.0,1.180154,1.545664,1.824121,59.0,6.0,8.0
yarca,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,160.0,142.0,1.055728,1.545184,1.631294,120.0,7.0,23.0
tolhuin,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,5,373.0,200.0,1.184478,1.534011,1.817002,57.0,8.0,9.0
fsa,0.0,0.0,0.0,0.0,22.0,2.0,0.0,0.0,4.0,1.0,...,0.0,6,321.0,232.0,1.033974,1.532793,1.584868,145.0,9.0,26.0
malpegue,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3,264.0,170.0,1.128838,1.523059,1.719288,77.0,10.0,16.0


Veamos qué performance tiene usando 1000, 2000, 3000, y así...

In [9]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
from contrastes.text import tokenize

liw_vectorizer = CountVectorizer(
    tokenizer=tokenize,
    vocabulary=word_df.index)

X_train = liw_vectorizer.fit_transform(df_train["text"])
print("Vectorizing")
X_test = liw_vectorizer.transform(df_test["text"])

Vectorizing
CPU times: user 9min 44s, sys: 556 ms, total: 9min 45s
Wall time: 9min 45s


Ya las tenemos vectorizadas en el orden esperado!

In [10]:
from sklearn.preprocessing import LabelEncoder

province_encoder = LabelEncoder()

province_encoder.fit(df_train["provincia"].values)

y_train = province_encoder.transform(df_train["provincia"].values)
y_test = province_encoder.transform(df_test["provincia"].values)

In [11]:
%%time
from contrastes.classifiers import fit_classifiers

num_words_to_fit = list(range(250, 5000, 250)) + list(range(5000, 20000, 500))

params = {"max_iter": 7000}

ret = fit_classifiers(X_train, y_train, X_test, y_test, 
                      province_encoder=province_encoder, clf_params=params,
                      range_num_words=num_words_to_fit, num_jobs=10)

Classifier params: {'multi_class': 'multinomial', 'solver': 'saga', 'penalty': 'l2', 'max_iter': 7000}
Entrenando con 750 palabras
Entrenando con 250 palabras
Entrenando con 1250 palabras
Entrenando con 2250 palabras
Entrenando con 2750 palabras
Entrenando con 3250 palabras
Entrenando con 3750 palabras
Entrenando con 1750 palabras
Entrenando con 4750 palabras
Entrenando con 4250 palabras
250   palabras ----> accuracy 59.48 mean distance 306.7484
Entrenando con 500 palabras
750   palabras ----> accuracy 69.60 mean distance 232.472
Entrenando con 1000 palabras
1250  palabras ----> accuracy 73.80 mean distance 228.4808
Entrenando con 1500 palabras
1750  palabras ----> accuracy 76.20 mean distance 207.0176
Entrenando con 2000 palabras
2750  palabras ----> accuracy 76.56 mean distance 196.4944
Entrenando con 3000 palabras
2250  palabras ----> accuracy 76.12 mean distance 186.1324
Entrenando con 2500 palabras
3250  palabras ----> accuracy 77.16 mean distance 178.4376
Entrenando con 3500 pala

In [12]:
for r in ret:
    num_words = r["num_words"]
    acc = r["accuracy"]
    md = r["mean_distance"]
    print("{:<5} palabras ----> accuracy {:.2f} mean distance {}".format(
        num_words, acc*100, md
    ))

250   palabras ----> accuracy 59.48 mean distance 306.7484
500   palabras ----> accuracy 67.24 mean distance 253.02
750   palabras ----> accuracy 69.60 mean distance 232.472
1000  palabras ----> accuracy 71.96 mean distance 245.748
1250  palabras ----> accuracy 73.80 mean distance 228.4808
1500  palabras ----> accuracy 74.72 mean distance 218.8384
1750  palabras ----> accuracy 76.20 mean distance 207.0176
2000  palabras ----> accuracy 76.28 mean distance 203.7484
2250  palabras ----> accuracy 76.12 mean distance 186.1324
2500  palabras ----> accuracy 76.16 mean distance 184.4612
2750  palabras ----> accuracy 76.56 mean distance 196.4944
3000  palabras ----> accuracy 77.00 mean distance 179.2064
3250  palabras ----> accuracy 77.16 mean distance 178.4376
3500  palabras ----> accuracy 78.12 mean distance 171.3148
3750  palabras ----> accuracy 78.16 mean distance 172.1536
4000  palabras ----> accuracy 78.56 mean distance 168.8024
4250  palabras ----> accuracy 78.76 mean distance 166.3432
4

In [ ]:
import pickle

pickle.dump(ret, open("res_iv_personas.pkl", "wb"))